# **5. Tips**
--------

## **Maximum Likelihood Estimation(MLE)**
학습을 반복적으로 진행하면서, 예측값의 확률 분포를 정답 확률 분포와 비슷하게 만들어 나가는 과정(Loss 설정, Cross Entropy).

**최대 우도 추정**    
**❓ Likelihood(우도)란?**   
<br>
p(data | distribution) = Probabilty   
p(distribution | data) = Likelihood   
<br>
즉, 입력 data가 고정되어 있을 때, 분포가 변화함.  
어떤 분포가 가장 data를 잘 설명하는가?

Parameter를 ϴ라고 할 때
$$ L(ϴ) = p(X|ϴ) $$
로 표현할 수 있다.

**❓ 그렇다면 MLE란?**   
ϴ를 추정하는 방법.   
Sample을 모듀 평균 값으로 지정해 Likelihood가 가장 큰 지점을 찾는다.   

예로, 압정이 있다고 치면
<br>
뾰족한 곳이 위로 = Class 1   
뾰족한 곳이 바닥 쪽으로 = Class 2    
<br>
로 나타낼 수 있다. 이런 확률 분포를 이항 분호, 베르누이 분포라고 한다.   
<br>

관찰 값이 있을 때, 압정이 Class 1이 되는 확률 분포, 압정이 Class 2r가 되는 확률 분포를 알고싶다!! 👉 MLE를 통해 예측   
<br>

즉, Observation을 가장 잘 설명하는 parameter(Observation이 Local Maxima가 되는!)를 찾아내는 과정 = MLE

그 점을 어떻게 찾죠?   
기울기를 구합니다.    
➡ Gradient Ascent 수행    
➡ Gradien Descent로 수행해도 과정은 같다   




## **Overfitting**
**[지난 이야기]**   
Gradient desent를 통한 최적화로 주어진 data를 가장 잘 설명하는 parameter를 찾게 되었다.   
하지만, MLE는 숙명적으로 Overfitting이 발생할 수 밖에 없게 되는데...   

**❓ Overffiting이란?**   
학습 data에 과도하게 경계선이 fitting 되버린 상황을 말함.
<br>

### **Overffiting 발생 여부 확인**
보통 dataset을 Training set, Test set으로 나눈다.(8, 2)   
더 나아가, Training set, Validation set, Test set(6, 2, 2)로 나누기도 한다.
<br>

**⚡Training set**:   
모델의 학습을 위한 dataset   
**⚡Validation set**:    
모델을 overffiting이 일어날 때 까지 학습한 후, Overfitting이 일어나기 바로 이전 지점(Baseline)에 해당하는 epoch 찾기 위해 사용되는 dataset.    
**⚡Test set**:     
모델 성능 평가를 위한 dataset. 전 단꼐에서 구한 epoch까지 진행, 학습에 관여하지 않음.    

Training set, Validation set, Test set 모두 비슷한 확률 분포를 가졌기 때문에, Training set이 오버피팅 되었는지 Validation set, Test set을 가지고 확인할 수 있다.
<br>

하지만 각각의 dataset을 가지고 너무 많이 돌리면, 모든 dataset들에 대해 overfitting이 발생할 수 있으니 주의!
<br><br>

### **Overfitting을 어떻게 최소화 할까?**
- More data
    - Data 수가 적을수록 편향된 결과를 얻음
- Less features
    - feature 수가 많아 복잡해질 경우에도 생길 가능성 높아짐
- **⭐ Regularization**
<br>

## **Regularization 방법**
- Early Stopping
    - Validation loss가 더 이상 낮아지지 않을 때 그냥 멈추기
- Reducing Network Size
- Weight Decay
    - Weight의 크기를 제한
- **⭐Dropout**
- **⭐Batch Nomalization**

<br>

### **Basic Approach to Train DNN**
1. Make a neural network architecture.
2. Train and check that model is over-fitted(overfitting 일어날 때 까지)   
        a. If it is not, increase th model size(deeper and wider).   
        b. If it is, add regularization, such as drop-out, batch-nomalization.   
3. Repeat from step-2   

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

### Training and Test Dataset

In [9]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]]) # |x_train| = (m, 3)
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0]) #|y_traini| = (m,)

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

### Model

In [11]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

### Training

In [1]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(1, nb_epochs):
        prediction = model(x_train)

        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


### Test(Validation)


In [27]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    print(prediction)
    predicted_classes = prediction.max(1)[1]
    print(prediction.max(1))
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost {:.6f}'.format(correct_count/len(y_test)* 100,  cost.item()))


### Train

In [16]:
train(model, optimizer, x_train, y_train)

Epoch    1/20 Cost: 0.825861
Epoch    2/20 Cost: 0.822643
Epoch    3/20 Cost: 0.819480
Epoch    4/20 Cost: 0.816369
Epoch    5/20 Cost: 0.813311
Epoch    6/20 Cost: 0.810303
Epoch    7/20 Cost: 0.807343
Epoch    8/20 Cost: 0.804431
Epoch    9/20 Cost: 0.801565
Epoch   10/20 Cost: 0.798744
Epoch   11/20 Cost: 0.795967
Epoch   12/20 Cost: 0.793232
Epoch   13/20 Cost: 0.790539
Epoch   14/20 Cost: 0.787887
Epoch   15/20 Cost: 0.785274
Epoch   16/20 Cost: 0.782700
Epoch   17/20 Cost: 0.780164
Epoch   18/20 Cost: 0.777664
Epoch   19/20 Cost: 0.775200


In [28]:
test(model, optimizer, x_test, y_test)

tensor([[-1.1730, -0.4375,  2.0221],
        [-1.3396, -0.4462,  2.6016],
        [-0.9965, -0.3337,  1.7039]], grad_fn=<AddmmBackward0>)
torch.return_types.max(
values=tensor([2.0221, 2.6016, 1.7039], grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 2]))
Accuracy: 100.0% Cost 0.121348


## Learning Rate


learning rate가 너무 크면 diverge하면서 cost가 점점 늘어난다.(overshooting)

In [29]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 1e5)

train(model, optimizer, x_train, y_train)
#Cost가 값이 커졌어요

Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 1100707.250000
Epoch    3/20 Cost: 2482261.250000
Epoch    4/20 Cost: 664769.812500
Epoch    5/20 Cost: 1668198.750000
Epoch    6/20 Cost: 748657.875000
Epoch    7/20 Cost: 1353832.250000
Epoch    8/20 Cost: 1790073.750000
Epoch    9/20 Cost: 917894.812500
Epoch   10/20 Cost: 989687.125000
Epoch   11/20 Cost: 990845.375000
Epoch   12/20 Cost: 1585082.375000
Epoch   13/20 Cost: 1265073.750000
Epoch   14/20 Cost: 1149144.875000
Epoch   15/20 Cost: 589766.937500
Epoch   16/20 Cost: 689678.500000
Epoch   17/20 Cost: 983032.875000
Epoch   18/20 Cost: 1265073.750000
Epoch   19/20 Cost: 1686644.750000


Learning rate가 너무 작으면 cost가 거의 줄어들지 않는다.

In [30]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-10)

train(model, optimizer, x_train, y_train)

Epoch    1/20 Cost: 1.341574
Epoch    2/20 Cost: 1.341574
Epoch    3/20 Cost: 1.341574
Epoch    4/20 Cost: 1.341574
Epoch    5/20 Cost: 1.341574
Epoch    6/20 Cost: 1.341574
Epoch    7/20 Cost: 1.341574
Epoch    8/20 Cost: 1.341574
Epoch    9/20 Cost: 1.341574
Epoch   10/20 Cost: 1.341574
Epoch   11/20 Cost: 1.341574
Epoch   12/20 Cost: 1.341574
Epoch   13/20 Cost: 1.341574
Epoch   14/20 Cost: 1.341574
Epoch   15/20 Cost: 1.341574
Epoch   16/20 Cost: 1.341574
Epoch   17/20 Cost: 1.341574
Epoch   18/20 Cost: 1.341574
Epoch   19/20 Cost: 1.341574


적절한 숫자로 시작해 발산하면 작게, Cost가 줄지 않으면 크게 조정합시다

In [31]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-1)
#Tip
#일단 1e-1부터 해서 어 괜찮네? 어 발산하네? 이런거 확인하고 조금씩 조정하는 것 추천

train(model, optimizer, x_train, y_train)

Epoch    1/20 Cost: 2.939317
Epoch    2/20 Cost: 1.887239
Epoch    3/20 Cost: 1.055398
Epoch    4/20 Cost: 0.936401
Epoch    5/20 Cost: 0.917493
Epoch    6/20 Cost: 0.911811
Epoch    7/20 Cost: 0.906384
Epoch    8/20 Cost: 0.901102
Epoch    9/20 Cost: 0.895959
Epoch   10/20 Cost: 0.890947
Epoch   11/20 Cost: 0.886062
Epoch   12/20 Cost: 0.881298
Epoch   13/20 Cost: 0.876650
Epoch   14/20 Cost: 0.872114
Epoch   15/20 Cost: 0.867685
Epoch   16/20 Cost: 0.863359
Epoch   17/20 Cost: 0.859132
Epoch   18/20 Cost: 0.855000
Epoch   19/20 Cost: 0.850961


## Data Processing

In [2]:
# Regression 문제
x_train = torch.FloatTensor([[73, 80, 75],
                            [93, 88, 93],
                            [89, 91, 90],
                            [96, 98, 100],
                            [73, 66, 70]]) #|x_train| = (m, 3)
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]]) #|y_train|=(m,))

In [3]:
# 정규 표준화로 전처리를 해 준다.
# Gradient Descent 기반의 최적화 알고리즘들은
#데이터의 스케일에 민감하다.
#정규화를 통해 입력 데이터의 스케일을 조절하면
#학습이 안정적으로 이루어질 가능성이 높아진다.

mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) /sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


### Training with Preprocessed Data
학습이 더 잘 되는 것을 볼 수 있다.

In [4]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)

    def forward(self, x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()

optimizer = optim.SGD(model.parameters(), lr = 1e-1)

In [7]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(1, nb_epochs):
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


In [6]:
train(model, optimizer, norm_x_train, y_train) #아니 망했는데...?

Epoch    1/20 Cost: -0.000000
Epoch    2/20 Cost: -0.000000
Epoch    3/20 Cost: -0.000000
Epoch    4/20 Cost: -0.000000
Epoch    5/20 Cost: -0.000000
Epoch    6/20 Cost: -0.000000
Epoch    7/20 Cost: -0.000000
Epoch    8/20 Cost: -0.000000
Epoch    9/20 Cost: -0.000000
Epoch   10/20 Cost: -0.000000
Epoch   11/20 Cost: -0.000000
Epoch   12/20 Cost: -0.000000
Epoch   13/20 Cost: -0.000000
Epoch   14/20 Cost: -0.000000
Epoch   15/20 Cost: -0.000000
Epoch   16/20 Cost: -0.000000
Epoch   17/20 Cost: -0.000000
Epoch   18/20 Cost: -0.000000
Epoch   19/20 Cost: -0.000000
